In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import numpy.random as npr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec  # Import for custom grid layout
from scipy import stats
from sklearn.model_selection import KFold
import pickle
import ssm

from notebooks.imports import *
from config import dir_config, main_config
from src.utils import pmf_utils, plot_utils, glm_hmm_utils


In [ ]:
compiled_dir = Path(dir_config.data.compiled)
processed_dir = Path(dir_config.data.processed)

#### utils

In [ ]:
def get_train_test_split(n_trials, target_fold, k_folds):
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=1)
    splits = [(train_idx, test_idx) for train_idx, test_idx in kf.split(np.arange(n_trials))]
    return splits[target_fold]

In [ ]:


def psychometric_fit(stimulus, choices, expected_choice_prob, ax, color, label, n_sample = 10):
    
    data = {
        "signed_coherence": np.array(stimulus)*100,
        "choice": choices
    }
    x_data, y_data, _, x_model, y_model = pmf_utils.get_psychometric_data(data)
    
    x_model_hat, y_model_hat = np.full((n_sample,len(x_model)),np.nan), np.full((n_sample,len(y_model)),np.nan)

    x_model_hat, y_model_hat = np.full((n_sample,len(x_model)),np.nan), np.full((n_sample,len(y_model)),np.nan)  
    for idx_sample in range(n_sample):
        data_fitted = {
            "signed_coherence": np.array(stimulus)*100,
            "choice": npr.binomial(1, expected_choice_prob)
        }
        x_data, y_data_hat, _, x_model_hat[idx_sample,:], y_model_hat[idx_sample,:] = pmf_utils.get_psychometric_data(data_fitted)

    ax.plot(x_data, y_data,'o', color=color)
    ax.plot(x_model, y_model, color=color, label=label)
    ax.plot(np.mean(x_model_hat, axis=0), np.mean(y_model_hat, axis=0), color = color, linestyle='--')
    ax.fill_between(np.mean(x_model_hat, axis=0), 
                    np.mean(y_model_hat, axis=0) - np.std(y_model_hat, axis=0), 
                    np.mean(y_model_hat, axis=0) + np.std(y_model_hat, axis=0),
                    color=color, alpha=0.3)

    ax.set_xlim(min(x_data),max(x_data))
    ax.set_xlabel('Coherence')
    ax.set_ylabel('choices toRF')
    ax.set_title('Psychometric fits',fontsize=15)
    ax.legend()


In [ ]:

def plot_model_fits(model, choices, input, stimulus, mask, n_states, session_name, task_switch, trial_indices=None):
    transition_matrix = model.transitions.params
    transition_matrix = np.exp(transition_matrix)[0]
    weights = -model.observations.params
    posterior_probs = model.expected_states(data=choices, input=input, mask=np.array(mask).reshape(-1,1))[0]

    fig = plt.figure(figsize=(14, 4))  # Wider figure for better spacing
    gs = gridspec.GridSpec(1, 4, width_ratios=[1, 1, 1, 2])  # Merge last two subplots

    cols = ['#ff7f00', '#4daf4a', '#377eb8']  # Add more colors if needed for higher n_states
    plt.suptitle(session_name)
    # ----  First Subplot: Psychometric Curves ----
    ax1 = plt.subplot(gs[0])

    weighted_sum = np.sum(weights * input[None, :, :], axis=-1).T
    sigmoid_output = 1 / (1 + np.exp(-weighted_sum)) 
    glm_hmm_expected_choice_prob = np.sum(sigmoid_output * posterior_probs, axis=1, keepdims=True)
    
    equal_indices = np.where(mask)[0][:task_switch]
    unequal_indices = np.where(mask)[0][task_switch:]
    if trial_indices is not None:
        equal_indices = np.intersect1d(trial_indices, equal_indices)
        unequal_indices = np.intersect1d(trial_indices, unequal_indices)
    
    psychometric_fit(stimulus[equal_indices], choices[equal_indices], glm_hmm_expected_choice_prob[equal_indices], ax1, '#377eb8', label="Equal")
    psychometric_fit(stimulus[unequal_indices], choices[unequal_indices], glm_hmm_expected_choice_prob[unequal_indices], ax1, '#974810', label="Unequal")
    
    # ---- Second Subplot: GLM Weights ----
    ax2 = plt.subplot(gs[1])
    for k in range(n_states):
        ax2.plot(np.arange(input.shape[1]), weights[k][0], marker='o',
                color=cols[k], linestyle='-', lw=1.5, label=f"State {k+1}")

    ax2.tick_params(axis='y', labelsize=10)
    ax2.set_ylabel("GLM weight", fontsize=15)
    ax2.set_xlabel("covariate", fontsize=15)
    ax2.set_xticks(range(input.shape[1]))
    ax2.set_xticklabels(['stimulus', 'bias', 'previous choice', 'previous target'], fontsize=12, rotation=15)
    ax2.axhline(y=0, color="k", alpha=0.5, ls="--")
    ax2.legend()
    ax2.set_title("GLM weights", fontsize=15)

    # ---- Third Subplot: Transition Matrix ----
    ax3 = plt.subplot(gs[2])
    im = ax3.imshow(transition_matrix, vmin=-0.8, vmax=1, cmap='bone')
    for i in range(transition_matrix.shape[0]):
        for j in range(transition_matrix.shape[1]):
            ax3.text(j, i, str(np.around(transition_matrix[i, j], decimals=2)), 
                    ha="center", va="center", color="k", fontsize=12)

    ax3.set_xlim(-0.5, n_states - 0.5)
    ax3.set_ylim(n_states - 0.5, -0.5)
    ax3.set_xticks(range(n_states))
    ax3.set_yticks(range(n_states))
    ax3.set_xlabel("state t+1", fontsize=15)
    ax3.set_ylabel("state t", fontsize=15)
    ax3.set_title("Generative transition matrix", fontsize=15)

    # ---- Fourth (Merged) Subplot: Posterior Probabilities ----
    ax4 = plt.subplot(gs[3])  # Merged across two columns
    for k in range(n_states):
        ax4.plot(posterior_probs[mask, k], label=f"State {k + 1}", lw=2, color=cols[k])

    ax4.set_ylim(-0.01, 1.01)
    ax4.set_yticks([0, 0.5, 1])
    ax4.tick_params(axis='y', labelsize=10)
    ax4.set_xlabel("trial #", fontsize=15)
    ax4.set_ylabel("p(state)", fontsize=15)
    ax4.axvline(x=task_switch, color="k", alpha=0.5, ls="--")
    ax4.legend()
    ax4.set_title("Posterior Probabilities", fontsize=15)


    plt.tight_layout()
    plt.show()


#### Load Data

In [ ]:
with open(Path(processed_dir, 'glm_hmm_all_trials.pkl'), 'rb') as f:
    glm_hmm = pickle.load(f)

session_metadata = pd.read_csv(Path(compiled_dir,'sessions_metadata.csv'))

#### Best state

In [ ]:
for n_state in range(glm_hmm["session_wise"]['test_ll'].shape[1]):
    print(f"Test likelihood for {n_state+2} states: {np.mean(glm_hmm['session_wise']['test_ll'][:,n_state,:])}")
best_state_idx = np.argmax(np.mean(glm_hmm["session_wise"]['test_ll'],axis=(0,2))) 
best_state = best_state_idx + 2
print(f"Best state is {best_state}")

#### Best fold for each session

In [ ]:
best_fold_session_wise = []
for session in range(glm_hmm["session_wise"]['test_ll'].shape[0]):
    best_fold_session_wise.append(np.argmax(glm_hmm["session_wise"]["test_ll"][session,best_state_idx,:]))

#### Get train/test  split for each session

In [ ]:
train_fold, test_fold = {}, {}
for idx_session, session in enumerate(session_metadata['session_id']):
    n_trials = len(glm_hmm["session_wise"]["data"][session]["choices"])
    best_fold = best_fold_session_wise[idx_session]
    train_fold[session], test_fold[session] = get_train_test_split(n_trials=n_trials, target_fold=best_fold, k_folds=5)

## Model fits and data recovery (train and test)

### Train Dataset

#### Psychometric Fits, GLM weights, transition matrix, p(state)

In [ ]:
for idx_session, session in enumerate(session_metadata['session_id']):
    
    train_idx = train_fold[session]
    
    model = glm_hmm["session_wise"]['models'][idx_session][best_state][best_fold_session_wise[idx_session]]
    choices = glm_hmm["session_wise"]["data"][session]["choices"].values.reshape(-1, 1)
    input = np.array(glm_hmm["session_wise"]["data"][session][["normalized_stimulus","bias","previous_choice","previous_target"]])


    stimulus = glm_hmm["session_wise"]["data"][session]["stimulus"]

    
    if glm_hmm["session_wise"]["data"][session]["mask"] is None:
        mask = None
    else:
        mask = glm_hmm["session_wise"]["data"][session]["mask"]
    mask = np.ones_like(choices, dtype=bool) if mask is None else mask

    prob_toRF = glm_hmm["session_wise"]["data"][session]["prob_toRF"]
    prob_toRF = prob_toRF[mask]
    task_switch = np.where((prob_toRF != 50) & ~np.isnan(prob_toRF))[0][0]
    plot_model_fits(model, choices, input, stimulus, mask, best_state, session, task_switch, trial_indices=train_idx)




### Test Dataset

In [ ]:

for idx_session, session in enumerate(session_metadata['session_id']):
    
    test_idx = test_fold[session]
    
    model = glm_hmm["session_wise"]['models'][idx_session][best_state][best_fold_session_wise[idx_session]]
    choices = glm_hmm["session_wise"]["data"][session]["choices"].values.reshape(-1, 1)
    input = np.array(glm_hmm["session_wise"]["data"][session][["normalized_stimulus","bias","previous_choice","previous_target"]])


    stimulus = glm_hmm["session_wise"]["data"][session]["stimulus"]

    
    mask = glm_hmm["session_wise"]["data"][session]["mask"]
    mask = np.ones_like(choices, dtype=bool) if mask is None else mask

    prob_toRF = glm_hmm["session_wise"]["data"][session]["prob_toRF"]
    prob_toRF = prob_toRF[mask]
    task_switch = np.where((prob_toRF != 50) & ~np.isnan(prob_toRF))[0][0]
    plot_model_fits(model, choices, input, stimulus, mask, best_state, session, task_switch, trial_indices=test_idx)
    break



# Fit on whole session and save model

In [ ]:
init_params['transition_matrices']

In [ ]:
# get parameter initializations for each session
init_params = {
    'glm_weights': {},
    'transition_matrices': {}
}
choices_session_wise, inputs_session_wise, masks_session_wise = [], [], []

for idx_session, session in enumerate(session_metadata['session_id']):
    init_params['glm_weights'][idx_session] = glm_hmm["session_wise"]['models'][idx_session][best_state][best_fold_session_wise[idx_session]].observations.params
    init_params['transition_matrices'][idx_session] = glm_hmm["session_wise"]['models'][idx_session][best_state][best_fold_session_wise[idx_session]].transitions.params
    
    
    choices_session_wise.append(glm_hmm["session_wise"]["data"][session]["choices"].values.reshape(-1, 1))
    inputs_session_wise.append(np.array(glm_hmm["session_wise"]["data"][session][["normalized_stimulus","bias","previous_choice","previous_target"]]))
    masks_session_wise.append(glm_hmm["session_wise"]["data"][session]["mask"].values.reshape(-1, 1))
    

In [ ]:
models_session, fit_ll_session = glm_hmm_utils.session_wise_fit(choices_session_wise, inputs_session_wise, masks=masks_session_wise,
                                                                                    n_sessions=len((session_metadata['session_id'])), init_params=init_params, n_states = best_state, n_iters= 1000)

models_session_wise, fit_lls_session_wise ={},{}
for idx_session, session_id in enumerate(session_metadata['session_id']):
    models_session_wise[session_id] = models_session[idx_session]
    fit_lls_session_wise[session_id] = fit_ll_session[idx_session]

In [ ]:
for idx_session, session in enumerate(session_metadata['session_id']):
        
    model = models_session_wise[session]
    choices = glm_hmm["session_wise"]["data"][session]["choices"].values.reshape(-1, 1)
    input = np.array(glm_hmm["session_wise"]["data"][session][["normalized_stimulus","bias","previous_choice","previous_target"]])


    stimulus = glm_hmm["session_wise"]["data"][session]["stimulus"]

    
    mask = glm_hmm["session_wise"]["data"][session]["mask"]
    mask = np.ones_like(choices, dtype=bool) if mask is None else mask
    
    prob_toRF = glm_hmm["session_wise"]["data"][session]["prob_toRF"]
    prob_toRF = prob_toRF[mask]
    task_switch = np.where((prob_toRF != 50) & ~np.isnan(prob_toRF))[0][0]
    plot_model_fits(model, choices, input, stimulus, mask, best_state, session, task_switch)

In [ ]:
# store data and models for session-wise
session_wise_fits = {
    'models': models_session_wise,
    'fit_lls': fit_lls_session_wise,
    'data': glm_hmm["session_wise"]['data'],
}


best_idx = glm_hmm["global"]["fits_lls_glm_hmm"][best_state].index(max(glm_hmm["global"]["fits_lls_glm_hmm"][best_state]))
global_fits = {
    'model': glm_hmm["global"]["models"][best_state][best_idx],
    'fit_ll': max(glm_hmm["global"]["fits_lls_glm_hmm"][best_state][best_idx]),
    'data': glm_hmm["global"]["data"]
}

models_and_data = {
    'global': global_fits,
    'session_wise': session_wise_fits,
}

with open(Path(processed_dir, f'glm_hmm_all_trials_final.pkl'), 'wb') as f:
    pickle.dump(models_and_data, f)